In [1]:
import pandas as pd
import numpy as np
#from simple_function import *
import os
#from faiss_cluster import *
#from drawing_plot import *
import pyranges as pr
import PyWGCNA
import os
#from gprofiler import GProfiler
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import OneHotEncoder
import networkx as nx
import igraph as ig
import leidenalg as la
import cupy
from sklearn.preprocessing import MinMaxScaler

In [2]:
##getting files

# !scp s4727702@bioinfr.server.science.uq.edu.au:/gtex project

# !scp s4727702@bioinfr.server.science.uq.edu.au:jupyter/jupyter/network_dataframe_majortissue/adjacency_matrix/*heart left* project/adj_matrix
# !scp s4727702@bioinfr.science.server.uq.edu.au:region/gencode.v29.annotation.gtf project

In [3]:
#### GTEX variant files """
files = os.listdir("project/gtex/")
first_f = True
for f in files:
    if '.egenes' in f:
        tissue = f.split('.v8')[0]
        #print(tissue)
        if first_f:
            gene_tested = pd.read_csv(f'project/gtex/{f}', sep='\t', compression='gzip', engine='c')
            gene_tested['Tissue'] = [tissue]*len(gene_tested)
            first_f=False
        else:
            reading = pd.read_csv(f'project/gtex/{f}', sep='\t', compression='gzip', engine='c')
            reading['Tissue'] = [tissue]*len(reading)
            gene_tested = pd.concat([gene_tested, reading], axis=0)
#gene_tested = gene_tested.query("qval <= 0.05")

In [4]:
var_summary = gene_tested.rename({
    'chr': 'Chromosome',
    'variant_pos': 'Start'
}, axis=1)
var_summary['End'] = var_summary['Start']
var_summary=pr.PyRanges(var_summary)

In [5]:
heart_left = os.listdir('project/adj_matrix/')

In [6]:
vertex_file = os.listdir('project/vertex_df/')
vertex_df = {}
for f in vertex_file:
    histone=f.split('_')[0]
    df = pd.read_feather(f'project/vertex_df/{f}')
    df.set_index('vertex ID', inplace=True)
    vertex_df[histone] = df

In [7]:
hm_dict = {}
for t in heart_left:
    histone = t.split('_')[0]
    adj = pd.read_csv(f'project/adj_matrix/{t}', compression='gzip', index_col=0)
    graph = ig.Graph.Weighted_Adjacency(adj, mode='upper', loops=False)
    hm_dict[histone] = graph

In [8]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

# 1 Histone - H3K4me3

In [10]:
import anndata as ann

In [11]:
peak_anndata = ann.read_h5ad('project/adata_hm_tissue_chr1.h5ad')

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
peak_anndata.layers['norm_within_assay'] = scaler.fit_transform(peak_anndata.layers['average_signal'].T).T

Setting up data to train

In [95]:
def name_to_region(list_name):
    chrom = [x.split(':')[0] for x in list_name]
    start = [x.split(':')[1].split('-')[0] for x in list_name]
    end = [x.split(':')[1].split('-')[1] for x in list_name]
    df = pd.DataFrame({'Chromosome': chrom, 'Start': start, 'End':end, 'name': list_name},)
    return df

In [94]:
hm_dict['H3K4me3'].get_vertex_dataframe()


,name
vertex ID,
0,chr1:777894-778893
1,chr1:778894-779893
2,chr1:816894-817893
3,chr1:826894-827893
4,chr1:827894-828893
...,...
2958,chr1:248862633-248863632
2959,chr1:248872633-248873632
2960,chr1:248873633-248874632


In [13]:
### set up graph data 
#edge id
edge_index = hm_dict['H3K4me3'].get_edge_dataframe()

node_idx = hm_dict['H3K4me3'].get_vertex_dataframe().loc[edge_index['source'].unique()]


#node id
node_idx = hm_dict['H3K4me3'].get_vertex_dataframe().index

#node label
node_labels = hm_dict['H3K4me3'].get_vertex_dataframe()

#node attributes
heart_left_sample = peak_anndata.obs.query('`Target of assay` == "H3K4me3" and Tissue.str.contains("heart left")').index #heart assay code
avg_signal = pd.DataFrame(peak_anndata.to_df('norm_within_assay').loc[heart_left_sample, node_labels.name].mean(0), columns=['norm_average_signal']).reset_index(drop=True) #average signal of all assay
peak_count = pd.DataFrame(peak_anndata.to_df('count').loc[heart_left_sample, node_labels.name].max(0), columns=['max_peaknumb']).reset_index(drop=True) #max number of peak in a region all of assays
node_attrs = pd.concat([node_labels, peak_count, avg_signal], axis=1)
node_attrs = node_attrs.drop('name', axis=1)

#edge id
edge_index = hm_dict['H3K4me3'].get_edge_dataframe()

print("Attributes:", node_attrs.shape)
print("Edges:", edge_index.shape)
print("Label:", node_labels.shape)

Attributes: (2963, 2)
Edges: (53894, 3)
Label: (2963, 1)


In [14]:
edge_idx = torch.tensor(edge_index.iloc[:, :2].to_numpy().transpose(), dtype=torch.long)
attrs = torch.tensor(node_attrs.to_numpy(), dtype=torch.float)
e_attr = torch.tensor(abs(edge_index.weight.to_numpy()), dtype=torch.float64)

In [15]:
data = Data(x=attrs, edge_index=edge_idx)
data.edge_weight=e_attr

Check isolated nodes -> remove

In [16]:
data.has_isolated_nodes()

True

Remove isolated nodes \
Transform into uindirected graph -> double the connection for other way \
Split training and val set

In [17]:
import torch_geometric.transforms as T
transform = T.Compose([T.remove_isolated_nodes.RemoveIsolatedNodes(),
                       T.ToUndirected(),
                       T.ToDevice(device),
                       T.RandomLinkSplit(
                                num_val=0.05,
                                num_test=0.1,
                                is_undirected=True,
                                add_negative_train_samples=False)])

In [18]:
train_data, val_data, test_data = transform(data)

print('Has isolated node: ', data.has_isolated_nodes())
print('train data', train_data)
print('val data', val_data)
print('test data', test_data)

Has isolated node:  False
train data Data(x=[2942, 2], edge_index=[2, 91622], edge_weight=[91622], edge_label=[45811], edge_label_index=[2, 45811])
val data Data(x=[2942, 2], edge_index=[2, 91622], edge_weight=[91622], edge_label=[5388], edge_label_index=[2, 5388])
test data Data(x=[2942, 2], edge_index=[2, 97010], edge_weight=[97010], edge_label=[10778], edge_label_index=[2, 10778])


In [19]:
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling
from tqdm import trange

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=100
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='dense')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")
    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [20]:
CUDA_LAUNCH_BLOCKING=1
model = Net(data.num_features, 124, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
model = train_link_predictor(model, train_data, val_data, optimizer, criterion, n_epochs=150)

test_auc = eval_link_predictor(model, test_data)
print(f"Test: {test_auc:.3f}")

Epoch: 010, Train Loss: 0.662, Val AUC: 0.785
Epoch: 020, Train Loss: 0.617, Val AUC: 0.786
Epoch: 030, Train Loss: 0.580, Val AUC: 0.800
Epoch: 040, Train Loss: 0.569, Val AUC: 0.785
Epoch: 050, Train Loss: 0.570, Val AUC: 0.793
Epoch: 060, Train Loss: 0.566, Val AUC: 0.797
Epoch: 070, Train Loss: 0.565, Val AUC: 0.809
Epoch: 080, Train Loss: 0.559, Val AUC: 0.824
Epoch: 090, Train Loss: 0.553, Val AUC: 0.837
Epoch: 100, Train Loss: 0.544, Val AUC: 0.842
Epoch: 110, Train Loss: 0.541, Val AUC: 0.842
Epoch: 120, Train Loss: 0.538, Val AUC: 0.840
Epoch: 130, Train Loss: 0.537, Val AUC: 0.844
Epoch: 140, Train Loss: 0.538, Val AUC: 0.841
Epoch: 150, Train Loss: 0.537, Val AUC: 0.840
Test: 0.831


In [21]:
heart_var = var_summary.df.query('Tissue.str.contains("Heart_Left") and Chromosome=="chr1"')

In [22]:
heart_var_pos = heart_var[['Chromosome', 'Start', 'End', 'variant_id', 'gene_id']]
heart_var_gene = heart_var[['gene_name', 'gene_chr', 'gene_start', 'gene_end', 'gene_id', 'variant_id']].rename({
    'gene_chr': 'Chromosome',
    'gene_start':'Start',
    'gene_end': 'End'
}, axis=1)

In [23]:
heart_var_pos=pr.PyRanges(heart_var_pos)
heart_var_gene=pr.PyRanges(heart_var_gene)

In [24]:
heart_var_gene

,gene_name,Chromosome,Start,End,gene_id,variant_id
0,WASH7P,chr1,14410,29553,ENSG00000227232.5,chr1_665098_G_A_b38
1,OR4G11P,chr1,62948,63887,ENSG00000240361.1,chr1_665098_G_A_b38
2,OR4F5,chr1,69091,70008,ENSG00000186092.4,chr1_807641_T_C_b38
3,RP11-34P13.15,chr1,135141,135895,ENSG00000268903.1,chr1_986007_G_A_b38
4,RP11-34P13.16,chr1,137682,137965,ENSG00000269981.1,chr1_1028281_C_T_b38
...,...,...,...,...,...,...
2136,TRIM58,chr1,247857199,247878205,ENSG00000162722.8,chr1_247577569_A_C_b38
2137,SH3BP5L,chr1,248810446,248826633,ENSG00000175137.10,chr1_248820444_A_G_b38
2138,ZNF672,chr1,248838210,248849517,ENSG00000171161.12,chr1_248471564_A_C_b38
2139,ZNF692,chr1,248850006,248859085,ENSG00000171163.15,chr1_248659210_A_G_b38


In [26]:
region_h3k4me3_heart = pr.PyRanges(name_to_region(node_labels.name.tolist()))

In [27]:
var_region = heart_var_pos.join(region_h3k4me3_heart)
var_region

,Chromosome,Start,End,variant_id,gene_id,Start_b,End_b,name
0,chr1,966227,966227,chr1_966227_C_G_b38,ENSG00000188976.10,965894,966893,chr1:965894-966893
1,chr1,982112,982112,chr1_982112_C_T_b38,ENSG00000187642.9,981894,982893,chr1:981894-982893
2,chr1,999666,999666,chr1_999666_G_A_b38,ENSG00000272512.1,998894,999893,chr1:998894-999893
3,chr1,1020217,1020217,chr1_1020217_G_T_b38,ENSG00000188157.14,1019894,1020893,chr1:1019894-1020893
4,chr1,1183198,1183198,chr1_1183198_G_A_b38,ENSG00000237330.2,1182894,1183893,chr1:1182894-1183893
...,...,...,...,...,...,...,...,...
94,chr1,236998167,236998167,chr1_236998167_C_T_b38,ENSG00000244020.2,236998089,236999088,chr1:236998089-236999088
95,chr1,244970360,244970360,chr1_244970360_G_C_b38,ENSG00000203666.12,244969633,244970632,chr1:244969633-244970632
96,chr1,246789097,246789097,chr1_246789097_C_G_b38,ENSG00000143653.9,246788633,246789632,chr1:246788633-246789632
97,chr1,247104366,247104366,chr1_247104366_G_T_b38,ENSG00000188295.14,247103633,247104632,chr1:247103633-247104632


In [28]:
gene_region = heart_var_gene.join(region_h3k4me3_heart)
gene_region

,gene_name,Chromosome,Start,End,gene_id,variant_id,Start_b,End_b,name
0,RP11-206L10.2,chr1,725885,778626,ENSG00000228327.3,chr1_14677_G_A_b38,777894,778893,chr1:777894-778893
1,RP11-206L10.9,chr1,778770,810060,ENSG00000237491.8,chr1_823322_C_G_b38,777894,778893,chr1:777894-778893
2,RP11-206L10.9,chr1,778770,810060,ENSG00000237491.8,chr1_823322_C_G_b38,778894,779893,chr1:778894-779893
3,FAM87B,chr1,817371,819837,ENSG00000177757.2,chr1_820846_C_A_b38,816894,817893,chr1:816894-817893
4,LINC01128,chr1,825138,859446,ENSG00000228794.8,chr1_841166_A_G_b38,826894,827893,chr1:826894-827893
...,...,...,...,...,...,...,...,...,...
2487,SH3BP5L,chr1,248810446,248826633,ENSG00000175137.10,chr1_248820444_A_G_b38,248825633,248826632,chr1:248825633-248826632
2488,ZNF672,chr1,248838210,248849517,ENSG00000171161.12,chr1_248471564_A_C_b38,248837633,248838632,chr1:248837633-248838632
2489,ZNF692,chr1,248850006,248859085,ENSG00000171163.15,chr1_248659210_A_G_b38,248858633,248859632,chr1:248858633-248859632
2490,PGBD2,chr1,248906196,248919946,ENSG00000185220.11,chr1_247919698_A_G_b38,248905633,248906632,chr1:248905633-248906632


In [29]:
var_gene_region = var_region.df.merge(gene_region.df, on=['gene_id', 'variant_id'])

In [30]:
var_gene_region_connection = var_gene_region[['name_x', 'name_y']]

In [31]:
var_gene_region_noloops=var_gene_region_connection.query('name_x != name_y')
var_gene_region_noloops.reset_index(drop=True, inplace=True)

In [32]:
node_labels.iloc[11]

name    chr1:958894-959893
Name: 11, dtype: object

In [33]:
var_gene_region_index = var_gene_region_noloops.merge(node_labels.reset_index(), left_on='name_x', right_on='name').merge(node_labels.reset_index(), left_on='name_y', right_on='name')

In [34]:
var_gene_region_index

,name_x,name_y,vertex ID_x,name_x,vertex ID_y,name_y
0,chr1:965894-966893,chr1:958894-959893,12,chr1:965894-966893,11,chr1:958894-959893
1,chr1:981894-982893,chr1:975894-976893,15,chr1:981894-982893,13,chr1:975894-976893
2,chr1:981894-982893,chr1:980894-981893,15,chr1:981894-982893,14,chr1:980894-981893
3,chr1:1019894-1020893,chr1:1024894-1025893,21,chr1:1019894-1020893,22,chr1:1024894-1025893
4,chr1:1019894-1020893,chr1:1032894-1033893,21,chr1:1019894-1020893,23,chr1:1032894-1033893
...,...,...,...,...,...,...
85,chr1:211382313-211383312,chr1:211416313-211417312,2559,chr1:211382313-211383312,2560,chr1:211416313-211417312
86,chr1:211630313-211631312,chr1:211830313-211831312,2563,chr1:211630313-211631312,2567,chr1:211830313-211831312
87,chr1:213982414-213983413,chr1:213985414-213986413,2592,chr1:213982414-213983413,2593,chr1:213985414-213986413
88,chr1:244969633-244970632,chr1:245051633-245052632,2917,chr1:244969633-244970632,2919,chr1:245051633-245052632


In [35]:
# from gtex: we have var -> gene
# var+hmloci and gene+hmloci
##### test var->region->gene #######

In [36]:
var_edge_index = var_gene_region_index[['vertex ID_x', 'vertex ID_y']]
var_edge_index.columns = ['source', 'target']
var_edge_index

,source,target
0,12,11
1,15,13
2,15,14
3,21,22
4,21,23
...,...,...
85,2559,2560
86,2563,2567
87,2592,2593
88,2917,2919


In [37]:
check_nodes = list(set(var_edge_index.source.tolist() + var_edge_index.target.tolist()))
check_x = node_attrs.loc[check_nodes,:].sort_index()

In [38]:
test_data.x

tensor([[1.0000, 0.0665],
        [2.0000, 0.3692],
        [1.0000, 0.0418],
        ...,
        [2.0000, 0.1249],
        [2.0000, 0.4891],
        [1.0000, 0.0740]], device='cuda:0')

In [39]:
var_region_gene = torch.tensor(var_edge_index.to_numpy().transpose(), dtype=torch.long)
var_region_gene_x = torch.tensor(check_x.to_numpy(), dtype=torch.float)
var_region_gene = Data(x=var_region_gene_x, edge_index=var_region_gene, device=device)
print(var_region_gene)

Data(x=[140, 2], edge_index=[2, 90], device=cuda)


In [40]:
output = model.encode(data.x, data.edge_index)
final = model.decode_all(output)
probabilities = torch.nn.functional.softmax(output, dim=0)

In [65]:
var_region_gene.edge

Data(x=[140, 2], edge_index=[2, 90], device=cuda)

In [66]:
final_np = final.cpu().numpy()

In [78]:
final_np_source = final_np[0]
final_np_target = final_np[1]
paired_edge = np.vstack((final_np_source, final_np_target)).T

In [81]:
paired_edge.shape

(4391658, 2)

In [98]:
len(index)

TypeError: object of type 'builtin_function_or_method' has no len()

In [84]:
index = var_edge_index.to_numpy()

In [90]:
count=0
for x in range(index.shape[0]):
    if index[x] in paired_edge:
        count+=1

In [91]:
count

90